In [2]:
import os
import requests
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [18]:
client_id = os.getenv("YELP_CLIENT_ID")
client_secret = os.getenv("YELP_API_KEY")
project_id = os.getenv("GCP_PROJECT_ID")
table_name = "businesses"
dataset_id = 'astoria'
def get_bearer_token(client_id, client_secret):
    # URL de la API de autenticación
    auth_url = "https://example.com/oauth/token"

    # Parámetros de la solicitud de autenticación
    auth_data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    # Realizar la solicitud de autenticación
    response = requests.post(auth_url, data=auth_data)

    # Obtener el token de portador de la respuesta
    token = response.json()["access_token"]

    return token

In [19]:
# try:
    # yelp_api = YelpAPI(api_key, timeout_s=3.0)
    # # search_results = yelp_api.search_query(args)
    # response = yelp_api.business_match_query(
    #     #name="Splash Cafe",
    #     #address1="197 Pomeroy Ave",
    #     #city="Pismo Beach",
    #     state="FL",
    #     country="US")
    # print(response)
# Realizar la solicitud HTTP
url = "https://api.yelp.com/v3/"+table_name+"/search?location=FL&sort_by=best_match"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer "+client_secret
}

response = requests.get(url, headers=headers)

print(response.text)
# finally:
#     # yelp_api.close()

{"businesses": [{"id": "1h8-r9cLeXxW99HhlRUnlA", "alias": "cvi-che-105-miami", "name": "CVI.CHE 105", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/uZ86aM2M7DJETF5JbEUQmw/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/cvi-che-105-miami?adjust_creative=SS9_C-4xTd48cYBvVQ-xbQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=SS9_C-4xTd48cYBvVQ-xbQ", "review_count": 5270, "categories": [{"alias": "peruvian", "title": "Peruvian"}, {"alias": "seafood", "title": "Seafood"}], "rating": 4.5, "coordinates": {"latitude": 25.775383, "longitude": -80.1884004}, "transactions": ["delivery"], "price": "$$", "location": {"address1": "105 NE 3rd Ave", "address2": "", "address3": "", "city": "Miami", "zip_code": "33132", "country": "US", "state": "FL", "display_address": ["105 NE 3rd Ave", "Miami, FL 33132"]}, "phone": "+13055773454", "display_phone": "(305) 577-3454", "distance": 10289.40979151124}, {"id": "UXHxLN3DcDGI57uDIfCuJA", "alias": "olds-havana-cuban-b

In [20]:
# Cargar la respuesta JSON en un objeto Python
data = response.json()
# seleccionar la columna "businesses"
businesses = data[table_name]
# Convertir el objeto JSON en un DataFrame de pandas
df = pd.json_normalize(businesses)
df = df.rename(columns={"coordinates.latitude": "latitude"})
df = df.rename(columns={"coordinates.longitude": "longitude"})
df = df.rename(columns={"location.address1": "address1"})
df = df.rename(columns={"location.address2": "address2"})
df = df.rename(columns={"location.address3": "address3"})
df = df.rename(columns={"location.city": "city"})
df = df.rename(columns={"location.zip_code": "zip_code"})
df = df.rename(columns={"location.country": "country"})
df = df.rename(columns={"location.state": "state"})
df = df.rename(columns={"location.display_address": "address"})
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,latitude,longitude,address1,address2,address3,city,zip_code,country,state,address
0,1h8-r9cLeXxW99HhlRUnlA,cvi-che-105-miami,CVI.CHE 105,https://s3-media3.fl.yelpcdn.com/bphoto/uZ86aM...,False,https://www.yelp.com/biz/cvi-che-105-miami?adj...,5270,"[{'alias': 'peruvian', 'title': 'Peruvian'}, {...",4.5,[delivery],...,25.775383,-80.188400,105 NE 3rd Ave,,,Miami,33132,US,FL,"[105 NE 3rd Ave, Miami, FL 33132]"
1,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,https://s3-media4.fl.yelpcdn.com/bphoto/OyMD-x...,False,https://www.yelp.com/biz/olds-havana-cuban-bar...,2340,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.5,"[delivery, pickup]",...,25.765594,-80.219238,1442 SW 8th St,,None,Miami,33135,US,FL,"[1442 SW 8th St, Miami, FL 33135]"
2,MfkZSdJ0w4-SOLJiEyHU5A,kyu-miami-miami,"KYU, Miami",https://s3-media2.fl.yelpcdn.com/bphoto/X8CG6v...,False,https://www.yelp.com/biz/kyu-miami-miami?adjus...,1899,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"[delivery, pickup]",...,25.800890,-80.200170,251 NW 25th St,None,,Miami,33127,US,FL,"[251 NW 25th St, Miami, FL 33127]"
3,SyVrRDKTGgVGNSNAbngxfw,glass-and-vine-miami,Glass and Vine,https://s3-media4.fl.yelpcdn.com/bphoto/5_9h4J...,False,https://www.yelp.com/biz/glass-and-vine-miami?...,1460,"[{'alias': 'newamerican', 'title': 'New Americ...",4.0,"[delivery, pickup]",...,25.726653,-80.241086,2820 McFarlane Rd,,,Miami,33133,US,FL,"[2820 McFarlane Rd, Miami, FL 33133]"
4,hZm7TunlrksQbgS0ssXbUg,versailles-miami-4,Versailles,https://s3-media1.fl.yelpcdn.com/bphoto/VaSpiR...,False,https://www.yelp.com/biz/versailles-miami-4?ad...,6245,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.0,"[delivery, pickup]",...,25.765039,-80.252798,3555 SW 8th St,,,Miami,33135,US,FL,"[3555 SW 8th St, Miami, FL 33135]"
5,Obt7Njta79-1CNtNwhNXiw,bunbury-miami-3,Bunbury,https://s3-media1.fl.yelpcdn.com/bphoto/uW2ngK...,False,https://www.yelp.com/biz/bunbury-miami-3?adjus...,843,"[{'alias': 'argentine', 'title': 'Argentine'},...",4.5,"[delivery, restaurant_reservation, pickup]",...,25.788458,-80.193363,55 NE 14th St,,,Miami,33132,US,FL,"[55 NE 14th St, Miami, FL 33132]"
6,yxPn5n5se97lnNYN5JcAIw,le-chick-miami-2,Le Chick,https://s3-media2.fl.yelpcdn.com/bphoto/FoAyja...,False,https://www.yelp.com/biz/le-chick-miami-2?adju...,1201,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,[delivery],...,25.799895,-80.200522,310 NW 24th St,,None,Miami,33127,US,FL,"[310 NW 24th St, Miami, FL 33127]"
7,opz-z4k-WK0KsyyR1Vwg5A,whisk-gourmet-food-and-catering-miami-2,WHISK Gourmet Food & Catering,https://s3-media2.fl.yelpcdn.com/bphoto/F8qi-P...,False,https://www.yelp.com/biz/whisk-gourmet-food-an...,1420,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"[delivery, pickup]",...,25.702922,-80.284768,7382 SW 56th Ave,,,Miami,33143,US,FL,"[7382 SW 56th Ave, Miami, FL 33143]"
8,bqTmkGBOUV6ReXWk1BVa2w,kush-by-spillover-miami,Kush by Spillover,https://s3-media1.fl.yelpcdn.com/bphoto/Pp9mpa...,False,https://www.yelp.com/biz/kush-by-spillover-mia...,976,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,[delivery],...,25.728450,-80.240630,2911 Grand Ave,,Ste 400 D,Miami,33133,US,FL,"[2911 Grand Ave, Ste 400 D, Miami, FL 33133]"
9,vpMIsdom05-oOBeF6ajESg,uchi-miami,Uchi,https://s3-media1.fl.yelpcdn.com/bphoto/Y9_NtG...,False,https://www.yelp.com/biz/uchi-miami?adjust_cre...,479,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.5,[],...,25.800531,-80.200013,252 NW 25th St,None,None,Miami,33127,US,FL,"[252 NW 25th St, Miami, FL 33127]"


In [23]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Convertir el DataFrame de pandas a un objeto pyarrow.Table
table = pa.Table.from_pandas(df)
# Guardar el objeto pyarrow.Table en formato Parquet
pq.write_table(table, table_name+".parquet")

# Guardar el DataFrame en formato Parquet
# df.to_parquet("businesses.parquet")

In [11]:
# Leer el DataFrame desde el archivo Parquet
df = pd.read_parquet(table_name+".parquet")
df.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'latitude', 'longitude', 'address1',
       'address2', 'address3', 'city', 'zip_code', 'country', 'state',
       'address'],
      dtype='object')

In [2]:
from google.oauth2 import service_account

# Cargar las credenciales desde un archivo JSON
credentials = service_account.Credentials.from_service_account_file('../../../credentials.json')

In [3]:
# Leer el DataFrame desde el archivo Parquet
df = pd.read_parquet(table_name+".parquet")
# guardar el dataframe en BigQuery
project_id = os.getenv("GCP_PROJECT_ID")
df = df.astype(str)
destination_table = f"{dataset_id}.yelp-{table_name}"
df.to_gbq(destination_table=destination_table, project_id=project_id, if_exists="replace")